2. EDA

In [ ]:
# 1. ИМПОРТ БИБЛИОТЕК
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import os

# Визуализация
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp
from plotly.offline import plot

# Машинное обучение (для кластеризации в RFM)
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler, QuantileTransformer
from sklearn.preprocessing import FunctionTransformer

In [ ]:
# Настройки отображения
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [ ]:
'''1. Загрузка данных'''
data = pd.read_csv('merged_sales_data.csv')

In [ ]:
plt.figure(figsize=(15,8))

plt.subplot(2,3,1)
sns.histplot(merged_data['SalesAmount'], kde=True, color='blue')
plt.title('Распределение SalesAmount')

plt.subplot(2,3,2)
sns.boxplot(data=merged_data, x='Group', y='SalesAmount')
plt.xticks(rotation=45)
plt.title('Продажи по группе регионов')

plt.subplot(2,3,3)
sns.boxplot(data=merged_data, x='Country', y='SalesAmount')
plt.xticks(rotation=45)
plt.title('Продажи по странам')

plt.subplot(2,3,4)
sns.countplot(data=merged_data, x='Region')
plt.xticks(rotation=45)
plt.title('Распределение по регионам')

plt.subplot(2,3,5)
sns.scatterplot(data=merged_data, x='YearlyIncome', y='SalesAmount', hue='Group', alpha=0.6)
plt.title('Доход vs Продажи (по группе регионов)')

plt.subplot(2,3,6)
sns.boxplot(data=merged_data, x='Category', y='SalesAmount', hue='Group')
plt.xticks(rotation=45)
plt.title('Продажи по категории и группе')

plt.tight_layout()
plt.show()

In [ ]:
# 📊 СОЗДАНИЕ ВИЗУАЛИЗАЦИЙ ДЛЯ EDA
print("📊 СОЗДАНИЕ ВИЗУАЛИЗАЦИЙ ДЛЯ EDA")
print("=" * 50)

# Создаем subplots
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
print("✅ Создана сетка графиков 2x3")

# 1. Динамика продаж по времени
if 'OrderDate' in df.columns and 'SalesAmount' in df.columns:
    monthly_sales = df.groupby(df['OrderDate'].dt.to_period('M'))['SalesAmount'].sum()
    axes[0,0].plot(monthly_sales.index.astype(str), monthly_sales.values, 
                  marker='o', linewidth=2, color='blue')
    axes[0,0].set_title('Динамика продаж по месяцам', fontsize=14, fontweight='bold')
    axes[0,0].set_xlabel('Месяц')
    axes[0,0].set_ylabel('Выручка (SalesAmount)')
    axes[0,0].tick_params(axis='x', rotation=45)
    axes[0,0].grid(True, alpha=0.3)
    print("✅ Построен график: Динамика продаж по месяцам")
else:
    axes[0,0].text(0.5, 0.5, 'Нет данных\nOrderDate или SalesAmount', 
                   ha='center', va='center', transform=axes[0,0].transAxes)
    axes[0,0].set_title('Динамика продаж по месяцам', fontsize=14, fontweight='bold')
    print("❌ Пропущен график: Отсутствуют OrderDate или SalesAmount")

# 2. Распределение по регионам
if 'Region' in df.columns:
    region_sales = df.groupby('Region')['SalesAmount'].sum().sort_values(ascending=False)
    axes[0,1].bar(range(len(region_sales)), region_sales.values, color='green', alpha=0.7)
    axes[0,1].set_title('Выручка по регионам', fontsize=14, fontweight='bold')
    axes[0,1].set_xlabel('Регион')
    axes[0,1].set_ylabel('Выручка')
    axes[0,1].set_xticks(range(len(region_sales)))
    axes[0,1].set_xticklabels(region_sales.index, rotation=45)
    print(f"✅ Построен график: Выручка по регионам ({len(region_sales)} регионов)")
else:
    axes[0,1].text(0.5, 0.5, 'Нет данных\nRegion', 
                   ha='center', va='center', transform=axes[0,1].transAxes)
    axes[0,1].set_title('Выручка по регионам', fontsize=14, fontweight='bold')
    print("❌ Пропущен график: Отсутствует Region")

# 3. Распределение по категориям продуктов
if 'Category' in df.columns:
    category_sales = df.groupby('Category')['SalesAmount'].sum()
    axes[0,2].pie(category_sales.values, labels=category_sales.index, 
                 autopct='%1.1f%%', startangle=90)
    axes[0,2].set_title('Распределение выручки по категориям', fontsize=14, fontweight='bold')
    print(f"✅ Построена круговая диаграмма: Категории продуктов ({len(category_sales)} категорий)")
else:
    axes[0,2].text(0.5, 0.5, 'Нет данных\nCategory', 
                   ha='center', va='center', transform=axes[0,2].transAxes)
    axes[0,2].set_title('Распределение выручки по категориям', fontsize=14, fontweight='bold')
    print("❌ Пропущена диаграмма: Отсутствует Category")

# 4. Распределение доходов клиентов
if 'YearlyIncome' in df.columns:
    axes[1,0].hist(df['YearlyIncome'], bins=20, alpha=0.7, edgecolor='black', color='orange')
    axes[1,0].set_title('Распределение годового дохода клиентов', fontsize=14, fontweight='bold')
    axes[1,0].set_xlabel('Годовой доход')
    axes[1,0].set_ylabel('Количество клиентов')
    print("✅ Построена гистограмма: Распределение доходов клиентов")
else:
    axes[1,0].text(0.5, 0.5, 'Нет данных\nYearlyIncome', 
                   ha='center', va='center', transform=axes[1,0].transAxes)
    axes[1,0].set_title('Распределение годового дохода клиентов', fontsize=14, fontweight='bold')
    print("❌ Пропущена гистограмма: Отсутствует YearlyIncome")

# 5. Распределение суммы продаж
if 'SalesAmount' in df.columns:
    axes[1,1].hist(df['SalesAmount'], bins=50, alpha=0.7, edgecolor='black', color='red')
    axes[1,1].set_title('Распределение суммы продаж', fontsize=14, fontweight='bold')
    axes[1,1].set_xlabel('Сумма продажи')
    axes[1,1].set_ylabel('Количество')
    print("✅ Построена гистограмма: Распределение суммы продаж")
else:
    axes[1,1].text(0.5, 0.5, 'Нет данных\nSalesAmount', 
                   ha='center', va='center', transform=axes[1,1].transAxes)
    axes[1,1].set_title('Распределение суммы продаж', fontsize=14, fontweight='bold')
    print("❌ Пропущена гистограмма: Отсутствует SalesAmount")

# 6. Топ продуктов по выручке
if 'ProductName' in df.columns:
    top_products = df.groupby('ProductName')['SalesAmount'].sum().nlargest(10)
    axes[1,2].barh(range(len(top_products)), top_products.values, color='purple', alpha=0.7)
    axes[1,2].set_yticks(range(len(top_products)))
    axes[1,2].set_yticklabels([label[:25] + '...' if len(label) > 25 else label 
                             for label in top_products.index])
    axes[1,2].set_title('Топ-10 продуктов по выручке', fontsize=14, fontweight='bold')
    axes[1,2].set_xlabel('Выручка')
    print(f"✅ Построен барчарт: Топ-10 продуктов по выручке")
else:
    axes[1,2].text(0.5, 0.5, 'Нет данных\nProductName', 
                   ha='center', va='center', transform=axes[1,2].transAxes)
    axes[1,2].set_title('Топ-10 продуктов по выручке', fontsize=14, fontweight='bold')
    print("❌ Пропущен барчарт: Отсутствует ProductName")

plt.tight_layout()
plt.savefig('eda_analysis.png', dpi=300, bbox_inches='tight')
print("💾 Графики сохранены в файл: eda_analysis.png")
plt.show()

# Дополнительные интерактивные визуализации с Plotly
print("\n📈 ИНТЕРАКТИВНЫЕ ВИЗУАЛИЗАЦИИ PLOTLY:")

if 'OrderDate' in df.columns and 'SalesAmount' in df.columns:
    daily_sales = df.groupby(df['OrderDate'].dt.date)['SalesAmount'].sum().reset_index()
    fig1 = px.line(daily_sales, x='OrderDate', y='SalesAmount', 
                  title='📈 Динамика ежедневных продаж',
                  labels={'OrderDate': 'Дата', 'SalesAmount': 'Выручка'})
    fig1.show()
    print("✅ Построен интерактивный график: Динамика ежедневных продаж")
else:
    print("❌ Пропущен интерактивный график: Отсутствуют OrderDate или SalesAmount")

if 'Region' in df.columns and 'SalesAmount' in df.columns:
    region_performance = df.groupby('Region').agg({
        'SalesAmount': 'sum',
        'CustomerKey': 'nunique',
        'OrderQuantity': 'sum'
    }).reset_index()
    
    fig2 = px.bar(region_performance, x='Region', y='SalesAmount',
                 title='🌍 Выручка по регионам',
                 color='CustomerKey',
                 labels={'SalesAmount': 'Выручка', 'CustomerKey': 'Количество клиентов'})
    fig2.show()
    print("✅ Построен интерактивный график: Выручка по регионам")
else:
    print("❌ Пропущен интерактивный график: Отсутствует Region или SalesAmount")

print(f"\n📊 ИТОГ ВИЗУАЛИЗАЦИЙ:")
print(f"   • Статических графиков: 6")
print(f"   • Интерактивных графиков: 2") 
print(f"   • Сохраненный файл: eda_analysis.png")
print("✅ ВИЗУАЛИЗАЦИИ УСПЕШНО СОЗДАНЫ!")

In [ ]:
def save_cleaned_data(df):
    """Сохранение очищенного датасета"""
    df.to_csv('cleaned_sales_data.csv', index=False, encoding='utf-8')
    print("✅ Очищенные данные сохранены в 'cleaned_sales_data.csv'")